In [1]:
import cobra
import re
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
import pandas as pd
import numpy as np
import xlrd
import re
import openpyxl
from metnet import pathway 
import sys
sys.path.append(r'./')
from COBRApy_ECM_funcation import *
from metnet import sbml_excel
import os
from copy import copy, deepcopy
import numpy as np
from cobra.io import write_sbml_model,save_json_model
from ec_code.cobrapy_ec_model_function import convert_to_irreversible,isoenzyme_split

In [2]:
from cobra.io import write_sbml_model,save_json_model,load_json_model,read_sbml_model

In [3]:
model_origin_name='./new_idea_AGGD/model/iML1515_new.json'
model_new='./new_idea_AGGD/model/iML1515_AG.json'
model_new2='./new_idea_AGGD/model/iML1515_GD.json'
model_new3='./new_idea_AGGD/model/iML1515_GD_26.json'

In [4]:
model_ab_json_new='./new_idea_AGGD/model/iML1515_AGGD_change.json'

## model_info

### A

In [4]:
model = cobra.io.load_json_model(model_origin_name)

In [5]:
model.reactions.get_by_id('EX_xyl__D_e').bounds=(-10,1000)
model.reactions.get_by_id('EX_glc__D_e').bounds=(0,1000)

### A knockout

In [6]:
#ptsG/manZ
model.reactions.get_by_id('GLCptspp')#.bounds=(0,0)

Reaction identifier,GLCptspp
Name,D-glucose transport via PEP:Pyr PTS (periplasm)
Memory address,0x01e7a3422588
Stoichiometry,glc__D_p + pep_c --> g6p_c + pyr_c D-Glucose + Phosphoenolpyruvate --> D-Glucose 6-phosphate + Pyruvate
GPR,( b2415 and b1101 and b2416 and b2417 ) or ( b2417 and b1621 and b2415 and b2416 ) or ( b1817 and...
Lower bound,0.0
Upper bound,1000.0


In [7]:
#PYK
PYK=['PYK','PYK2','PYK3','PYK4','PYK6']
for i in PYK:
    print(model.reactions.get_by_id(i))
    model.reactions.get_by_id(i).bounds=(0,0)

PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
PYK2: h_c + pep_c + udp_c --> pyr_c + utp_c
PYK3: gdp_c + h_c + pep_c --> gtp_c + pyr_c
PYK4: cdp_c + h_c + pep_c --> ctp_c + pyr_c
PYK6: dtdp_c + h_c + pep_c --> dttp_c + pyr_c


In [32]:
#pheA PPNDH
print(model.reactions.get_by_id('PPNDH'))
model.reactions.get_by_id('PPNDH').bounds=(0,0)
#feaB ALDD19xr NO KNOCK
print(model.reactions.get_by_id('ALDD19xr'))
model.reactions.get_by_id('ALDD19xr').bounds=(-1000,1000)

PPNDH: h_c + pphn_c --> co2_c + h2o_c + phpyr_c
ALDD19xr: h2o_c + nad_c + pacald_c --> 2.0 h_c + nadh_c + pac_c


In [9]:
#tyrA PPMD
print(model.reactions.get_by_id('PPND'))
model.reactions.get_by_id('PPND')

PPND: nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c


Reaction identifier,PPND
Name,Prephenate dehydrogenase
Memory address,0x01e7a2ded198
Stoichiometry,nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c Nicotinamide adenine dinucleotide + Prephenate --> 3-(4-Hydroxyphenyl)pyruvate + CO2 CO2 + Nicotinamide adenine dinucleotide - reduced
GPR,b2600
Lower bound,0.0
Upper bound,1000.0


In [10]:
#ACMANAptspp manZ
print(model.reactions.get_by_id('ACMANAptspp'))
model.reactions.get_by_id('ACMANAptspp').bounds=(0,0)

ACMANAptspp: acmana_p + pep_c --> acmanap_c + pyr_c


In [11]:
#MANptspp manZ
print(model.reactions.get_by_id('MANptspp'))
model.reactions.get_by_id('MANptspp').bounds=(0,0)

MANptspp: man_p + pep_c --> man6p_c + pyr_c


In [12]:
#FRUpts2pp manZ
print(model.reactions.get_by_id('FRUpts2pp'))
model.reactions.get_by_id('FRUpts2pp').bounds=(0,0)

FRUpts2pp: fru_p + pep_c --> f6p_c + pyr_c


In [13]:
#2DGLCptspp manZ
print(model.reactions.get_by_id('2DGLCptspp'))
model.reactions.get_by_id('2DGLCptspp').bounds=(0,0)

2DGLCptspp: 2dglc_p + pep_c --> 2doxg6p_c + pyr_c


### A add

In [30]:
hoxpacd_c=deepcopy(model.metabolites.get_by_id('4hoxpacd_e'))
hoxpacd_c.compartment='c'
hoxpacd_c.id='hoxpacd_c'
hoxpacd_c

Metabolite identifier,hoxpacd_c
Name,4-Hydroxyphenylacetaldehyde
Memory address,0x01e7a22cb8d0
Formula,C8H8O2
Compartment,c
In 0 reaction(s),


In [27]:
model.metabolites.get_by_id('4hoxpacd_e')

Metabolite identifier,4hoxpacd_e
Name,4-Hydroxyphenylacetaldehyde
Memory address,0x01e7a1fe15f8
Formula,C8H8O2
Compartment,e
In 2 reaction(s),"EX_4hoxpacd_e, 4HOXPACDtex"


In [24]:
model.metabolites.get_by_id('4hoxpacd_p')

Metabolite identifier,4hoxpacd_p
Name,4-Hydroxyphenylacetaldehyde
Memory address,0x01e7a205bf28
Formula,C8H8O2
Compartment,p
In 2 reaction(s),"4HOXPACDtex, TYROXDApp"


In [28]:
model.reactions.get_by_id('EX_4hoxpacd_e')

Reaction identifier,EX_4hoxpacd_e
Name,4-Hydroxyphenylacetaldehyde exchange
Memory address,0x01e7a376c828
Stoichiometry,4hoxpacd_e --> 4-Hydroxyphenylacetaldehyde -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [26]:
model.reactions.get_by_id('ALDD19xr')

Reaction identifier,ALDD19xr
Name,"Aldehyde dehydrogenase (phenylacetaldehyde, NAD)"
Memory address,0x01e7a3c5a0b8
Stoichiometry,h2o_c + nad_c + pacald_c --> 2.0 h_c + nadh_c + pac_c H2O H2O + Nicotinamide adenine dinucleotide + Phenylacetaldehyde --> 2.0 H+ + Nicotinamide adenine dinucleotide - reduced + Phenylacetic acid
GPR,b1385
Lower bound,0
Upper bound,0


In [ ]:
hpp_c

In [ ]:
hpp_c=model.metabolites.get_by_id('34hpp_c')
h_c=model.metabolites.get_by_id('h_c')
co2_c=model.metabolites.get_by_id('co2_c')
PDC=Reaction("PDC")
PDC.name='pyruvate decarboxylase'
PDC.subsystem='unknow'
PDC.lower_bound=0
PDC.upper_bound=1000
PDC.add_metabolites({
    hpp_c:-1.0,
    h_c:-1.0,
    co2_c:1.0,
    hoxpacd_c:1.0
})
PDC

In [33]:
Tyrosol_c=Metabolite('Tyrosol_c',
                formula='C8H10O2',
                name='Tyrosol',
                compartment='c'
                )

In [34]:
Tyrosol_e=Metabolite('Tyrosol_e',
                formula='C8H10O2',
                name='Tyrosol',
                compartment='e'
                )

In [35]:
EX_Tyrosol_e=Reaction("EX_Tyrosol_e")
EX_Tyrosol_e.name='Tyrosol_e exchange'
EX_Tyrosol_e.subsystem='unknow'
EX_Tyrosol_e.lower_bound=0
EX_Tyrosol_e.upper_bound=1000
EX_Tyrosol_e.add_metabolites({
    Tyrosol_e:-1.0
})
Tyrosol_c2e=Reaction("Tyrosol_c2e")
Tyrosol_c2e.name='4adcho_c2e'
Tyrosol_c2e.subsystem='unknow'
Tyrosol_c2e.lower_bound=-1000
Tyrosol_c2e.upper_bound=1000
Tyrosol_c2e.add_metabolites({
    Tyrosol_c:-1.0,
    Tyrosol_e:1.0
})
model.add_reactions([EX_Tyrosol_e])
model.add_reactions([Tyrosol_c2e])

In [36]:
phpyr_c=model.metabolites.get_by_id('phpyr_c')
phpyr_e=deepcopy(phpyr_c)
phpyr_e.compartment='e'
phpyr_e.id='phpyr_e'
phpyr_e

Metabolite identifier,phpyr_e
Name,Phenylpyruvate
Memory address,0x01e7a22d4908
Formula,C9H7O3
Compartment,e
In 0 reaction(s),


In [37]:
EX_phpyr_e=Reaction("EX_phpyr_e")
EX_phpyr_e.name='phpyr_e exchange'
EX_phpyr_e.subsystem='unknow'
EX_phpyr_e.lower_bound=-1000
EX_phpyr_e.upper_bound=1000
EX_phpyr_e.add_metabolites({
    phpyr_e:-1.0
})
phpyr_c2e=Reaction("phpyr_c2e")
phpyr_c2e.name='phpyr_c2e'
phpyr_c2e.subsystem='unknow'
phpyr_c2e.lower_bound=-1000
phpyr_c2e.upper_bound=1000
phpyr_c2e.add_metabolites({
    phpyr_c:-1.0,
    phpyr_e:1.0
})
model.add_reactions([EX_phpyr_e])
model.add_reactions([phpyr_c2e])

In [38]:
nad_c=model.metabolites.get_by_id('nad_c')
h_c=model.metabolites.get_by_id('h_c')
nadh_c=model.metabolites.get_by_id('nadh_c')
ADH=Reaction("ADH")
ADH.name='alcohol dehydrogenase'
ADH.subsystem='unknow'
ADH.lower_bound=0
ADH.upper_bound=1000
ADH.add_metabolites({
    hoxpacd_c:-1.0,
    nadh_c:-1.0,
    h_c:-1.0,
    nad_c:1.0,
    Tyrosol_c:1.0
})
ADH

Reaction identifier,ADH
Name,alcohol dehydrogenase
Memory address,0x01e7a22fe860
Stoichiometry,h_c + hoxpacd_c + nadh_c --> Tyrosol_c + nad_c H+ + 4-Hydroxyphenylacetaldehyde + Nicotinamide adenine dinucleotide - reduced --> Tyrosol + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0
Upper bound,1000


In [39]:
model.add_reactions([PDC])
model.add_reactions([ADH])

In [40]:
product = model.metabolites.get_by_id('Tyrosol_c')
demand= model.add_boundary(product, type='demand')  # add demand reaction as the objective
model.objective = demand

In [42]:
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds=(0.1,1000)

In [45]:
model.reactions.EX_tyr__L_e.bounds=(0.1,1000)

In [46]:
model.optimize()
model.summary()

IN FLUXES            OUT FLUXES      OBJECTIVES
-------------------  --------------  ------------------
xyl__D_e  10         h2o_e     28.4  DM_Tyrosol_c  3.96
o2_e       7.26      co2_e     13.5
nh4_e      1.18      h_e        1
pi_e       0.0965    tyr__L_e   0.1
so4_e      0.0252
k_e        0.0195
phpyr_e    0.0185
fe2_e      0.00161
mg2_e      0.000868
ca2_e      0.000521
cl_e       0.000521
cu2_e      7.09e-05
mn2_e      6.91e-05
zn2_e      3.41e-05
ni2_e      3.23e-05


In [47]:
model.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_xyl__D_e': 10,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_phpyr_e': 1000}

In [48]:
save_json_model(model,model_new)

In [49]:
write_sbml_model(model,model_new.split('.json')[0]+'.xml')

In [50]:
sbml_excel(model_new.split('.json')[0]+'.xml',model_new.split('.json')[0]+'.xlsx')

./new_idea_AGGD/model/iML1515_AG.xlsx


### B

In [1061]:
model = cobra.io.load_json_model(model_origin_name)

In [1062]:
model.reactions.get_by_id('EX_glc__D_e').bounds=(-10,1000)
model.reactions.get_by_id('EX_xyl__D_e').bounds=(0,1000)

### B knockout

In [1063]:
#xylA XYLT1
print(model.reactions.get_by_id('XYLI1'))
model.reactions.get_by_id('XYLI1').bounds=(0,0)
#ushA UDPGPpp
print(model.reactions.get_by_id('UDPGPpp'))
model.reactions.get_by_id('UDPGPpp').bounds=(0,0)
#tyrA PPND
print(model.reactions.get_by_id('PPND'))
# model.reactions.get_by_id('PPND').bounds=(0,0)
print(model.reactions.get_by_id('CHORM'))
# model.reactions.get_by_id('CHORM').bounds=(0,0)

XYLI1: xyl__D_c <=> xylu__D_c
UDPGPpp: h2o_p + udpg_p --> g1p_p + 2.0 h_p + ump_p
PPND: nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c
CHORM: chor_c --> pphn_c


In [1064]:
model.reactions.get_by_id('TYRTA').bounds=(0,1000)

In [1065]:
model.reactions.get_by_id('TYRPpp').bounds=(0,0)

In [1066]:
model.reactions.get_by_id('TYRtipp')

Reaction identifier,TYRtipp
Name,L-tyrosine transport out (periplasm)
Memory address,0x01e808160860
Stoichiometry,tyr__L_c --> tyr__L_p L-Tyrosine --> L-Tyrosine
GPR,b1473
Lower bound,0.0
Upper bound,1000.0


In [1067]:
Tyrosol_c=Metabolite('Tyrosol_c',
                formula='C8H10O2',
                name='Tyrosol',
                compartment='c'
                )
Tyrosol_e=Metabolite('Tyrosol_e',
                formula='C8H10O2',
                name='Tyrosol',
                compartment='e'
                )

In [1068]:
phpyr_c=model.metabolites.get_by_id('phpyr_c')
phpyr_e=deepcopy(phpyr_c)
phpyr_e.compartment='e'
phpyr_e.id='phpyr_e'
phpyr_e

Metabolite identifier,phpyr_e
Name,Phenylpyruvate
Memory address,0x01e8109a9b00
Formula,C9H7O3
Compartment,e
In 0 reaction(s),


In [1069]:
EX_phpyr_e=Reaction("EX_phpyr_e")
EX_phpyr_e.name='phpyr_e exchange'
EX_phpyr_e.subsystem='unknow'
EX_phpyr_e.lower_bound=-1000
EX_phpyr_e.upper_bound=1000
EX_phpyr_e.add_metabolites({
    phpyr_e:-1.0
})
phpyr_c2e=Reaction("phpyr_c2e")
phpyr_c2e.name='phpyr_c2e'
phpyr_c2e.subsystem='unknow'
phpyr_c2e.lower_bound=-1000
phpyr_c2e.upper_bound=1000
phpyr_c2e.add_metabolites({
    phpyr_c:-1.0,
    phpyr_e:1.0
})
model.add_reactions([EX_phpyr_e])
model.add_reactions([phpyr_c2e])

In [1070]:
EX_Tyrosol_e=Reaction("EX_Tyrosol_e")
EX_Tyrosol_e.name='Tyrosol_e exchange'
EX_Tyrosol_e.subsystem='unknow'
EX_Tyrosol_e.lower_bound=-1000
EX_Tyrosol_e.upper_bound=1000
EX_Tyrosol_e.add_metabolites({
    Tyrosol_e:-1.0
})
Tyrosol_c2e=Reaction("Tyrosol_c2e")
Tyrosol_c2e.name='Tyrosol_c2e'
Tyrosol_c2e.subsystem='unknow'
Tyrosol_c2e.lower_bound=-1000
Tyrosol_c2e.upper_bound=1000
Tyrosol_c2e.add_metabolites({
    Tyrosol_c:-1.0,
    Tyrosol_e:1.0
})
model.add_reactions([EX_Tyrosol_e])
model.add_reactions([Tyrosol_c2e])

In [1071]:
Salid_c=Metabolite('Salid_c',
                formula='C14H20O7',
                name='Salidroside',
                compartment='c'
                )

In [1072]:
Salid_e=Metabolite('Salid_e',
                formula='C14H20O7',
                name='Salidroside',
                compartment='e'
                )

In [1073]:
EX_Salid_e=Reaction("EX_Salid_e")
EX_Salid_e.name='Salid_e exchange'
EX_Salid_e.subsystem='unknow'
EX_Salid_e.lower_bound=0
EX_Salid_e.upper_bound=1000
EX_Salid_e.add_metabolites({
    Salid_e:-1.0
})
Salid_c2e=Reaction("Salid_c2e")
Salid_c2e.name='Salid_c2e'
Salid_c2e.subsystem='unknow'
Salid_c2e.lower_bound=-1000
Salid_c2e.upper_bound=1000
Salid_c2e.add_metabolites({
    Salid_c:-1.0,
    Salid_e:1.0
})
model.add_reactions([EX_Salid_e])
model.add_reactions([Salid_c2e])

In [1074]:
udp_c=model.metabolites.get_by_id('udp_c')
udpg_c=model.metabolites.get_by_id('udpg_c')
h_c=model.metabolites.get_by_id('h_c')
RXN_12370=Reaction("RXN_12370")
RXN_12370.name='tyrosol UDP-glucosyltransferase'
RXN_12370.subsystem='unknow'
RXN_12370.lower_bound=0
RXN_12370.upper_bound=1000
RXN_12370.add_metabolites({
    udpg_c:-1.0,
    Tyrosol_c:-1.0,
    h_c:1.0,
    udp_c:1.0,
    Salid_c:1.0
})
RXN_12370

Reaction identifier,RXN_12370
Name,tyrosol UDP-glucosyltransferase
Memory address,0x01e7f7f12c88
Stoichiometry,Tyrosol_c + udpg_c --> Salid_c + h_c + udp_c Tyrosol + UDPglucose --> Salidroside + H+ + UDP C9H11N2O12P2
GPR,
Lower bound,0
Upper bound,1000


In [1075]:
model.add_reactions([RXN_12370])

In [1076]:
model.reactions.get_by_id('EX_glc__D_e').bounds=(-10,1000)
model.reactions.get_by_id('EX_xyl__D_e').bounds=(0,1000)

In [1077]:
model.reactions.get_by_id('EX_tyr__L_e').bounds=(-1000,1000)

In [1078]:
product2 = model.metabolites.get_by_id('Salid_c')
demand2= model.add_boundary(product2, type='demand')  # add demand reaction as the objective
model.objective = demand2

In [1079]:
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds=(0.1,1000)

In [1080]:
model.reactions.EX_phpyr_e.bounds=(0.8,1000)

In [1081]:
model.optimize()
model.summary()

IN FLUXES             OUT FLUXES      OBJECTIVES
--------------------  --------------  ----------------
glc__D_e   10         h2o_e    22.2   DM_Salid_c  6.64
o2_e        8.38      co2_e     8.99
Tyrosol_e   6.64      h_e       1.71
nh4_e       1.07      phpyr_e   0.8
pi_e        0.0965
so4_e       0.0252
k_e         0.0195
tyr__L_e    0.0138
fe2_e       0.00161
mg2_e       0.000868
ca2_e       0.000521
cl_e        0.000521
cu2_e       7.09e-05
mn2_e       6.91e-05
zn2_e       3.41e-05
ni2_e       3.23e-05


In [1083]:
save_json_model(model,model_new3)
write_sbml_model(model,model_new3.split('.json')[0]+'.xml')
sbml_excel(model_new3.split('.json')[0]+'.xml',model_new3.split('.json')[0]+'.xlsx')

./new_idea_AGGD/model/iML1515_GD_26.xlsx


In [89]:
save_json_model(model,model_new2)
write_sbml_model(model,model_new2.split('.json')[0]+'.xml')
sbml_excel(model_new2.split('.json')[0]+'.xml',model_new2.split('.json')[0]+'.xlsx')

./new_idea_AGGD/model/iML1515_GD.xlsx


### AB

In [1084]:
model_a_name=model_new
model_b_name=model_new3

In [1085]:
model_a=load_json_model(model_a_name)
model_b=load_json_model(model_b_name)

In [1086]:
model_a.reactions.get_by_id('EX_glc__D_e').bounds=(0,1000)
model_a.reactions.get_by_id('EX_xyl__D_e').bounds=(-10,1000)
model_a.reactions.get_by_id('EX_phpyr_e').bounds=(-1000,0)
model_b.reactions.get_by_id('EX_Tyrosol_e').bounds=(-1000,0)
model_b.reactions.get_by_id('EX_tyr__L_e').bounds=(-1000,0)
model_b.reactions.get_by_id('EX_xyl__D_e').bounds=(0,1000)
model_b.reactions.get_by_id('EX_glc__D_e').bounds=(-10,1000)

In [1087]:
print("--------A  model----------")
print(len(model_a.reactions))
print(len(model_a.metabolites))
print(len(model_a.genes) )
print("--------B  model----------")
print(len(model_b.reactions))
print(len(model_b.metabolites))
print(len(model_b.genes) )

--------A  model----------
2720
1881
1516
--------B  model----------
2721
1882
1516


In [ ]:
# convert_to_irreversible(model_a)
# #split isoenzyme
# model_a = isoenzyme_split(model_a)
# #同工酶反应拆分
# save_json_model(model_a,model_a_name.split('/model/')[0]+'/model/reversed/Reverse'+model_a_name.split('/model/')[1])
# convert_to_irreversible(model_b)
# #split isoenzyme
# model_b = isoenzyme_split(model_b)
# #同工酶反应拆分
# save_json_model(model_b,model_b_name.split('/model/')[0]+'/model/reversed/Reverse'+model_b_name.split('/model/')[1])

In [94]:
model_a_name.split('/model/')[0]+'/model/reversed/Reverse'+model_a_name.split('/model/')[1]

'./new_idea_AGGD/model/reversed/ReverseiML1515_AG.json'

In [1088]:
model=deepcopy(model_a)
for i in model.reactions:
    i.id='A_'+i.id
for i in model.metabolites:
    i.id='A_'+i.id
    i.name='A_'+i.name
model_t=deepcopy(model_b)
# for i in model_t.genes:
#     i.id=i.id+'_B'
for i in model_t.reactions:
    i.id='B_'+i.id
for i in model_t.metabolites:
    i.id='B_'+i.id
    i.name='B_'+i.name

In [1089]:
for i in model_t.reactions:
    model.add_reaction(i)

In [1090]:
e_meta_rn={}
for meta in model_a.metabolites:
    if meta.compartment=='e' :
        meta_a=model.metabolites.get_by_id('A_'+meta.id)
        e_meta_rn[meta.id+"_1"]=Reaction('A_'+meta.id+'_con_1')
        e_meta_rn[meta.id+"_1"].name = meta.id+' Connection 1'
        e_meta_rn[meta.id+"_1"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_1"].lower_bound = 0. # This is the default
        e_meta_rn[meta.id+"_1"].upper_bound = 1000. # This is the default
        e_meta_rn[meta.id+"_1"].add_metabolites({
            meta_a:-1.0,
            meta:1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_1"]])
for meta in model_b.metabolites:
    if meta.compartment=='e' :
        meta_b=model_t.metabolites.get_by_id('B_'+meta.id)
        e_meta_rn[meta.id+"_2"]=Reaction('B_'+meta.id+'_con_2')
        e_meta_rn[meta.id+"_2"].name = meta.id+' Connection 2'
        e_meta_rn[meta.id+"_2"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_2"].lower_bound = -1000. # This is the default
        e_meta_rn[meta.id+"_2"].upper_bound = 0. # This is the default
        e_meta_rn[meta.id+"_2"].add_metabolites({
            meta_b:1.0,
            meta:-1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_2"]])

In [1091]:
model_origin_name='./new_idea_AGGD/model/iML1515_new.json'

In [1092]:
origin_model=cobra.io.load_json_model(model_origin_name)
origin_model.reactions.get_by_id('EX_xyl__D_e').bounds=(-10.0,1000)
origin_model.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_xyl__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [1093]:
model.medium

{'A_EX_pi_e': 1000.0,
 'A_EX_h_e': 1000.0,
 'A_EX_fe3_e': 1000.0,
 'A_EX_mn2_e': 1000.0,
 'A_EX_co2_e': 1000.0,
 'A_EX_fe2_e': 1000.0,
 'A_EX_xyl__D_e': 10,
 'A_EX_zn2_e': 1000.0,
 'A_EX_mg2_e': 1000.0,
 'A_EX_ca2_e': 1000.0,
 'A_EX_ni2_e': 1000.0,
 'A_EX_cu2_e': 1000.0,
 'A_EX_cobalt2_e': 1000.0,
 'A_EX_sel_e': 1000.0,
 'A_EX_h2o_e': 1000.0,
 'A_EX_nh4_e': 1000.0,
 'A_EX_mobd_e': 1000.0,
 'A_EX_so4_e': 1000.0,
 'A_EX_k_e': 1000.0,
 'A_EX_na1_e': 1000.0,
 'A_EX_o2_e': 1000.0,
 'A_EX_cl_e': 1000.0,
 'A_EX_tungs_e': 1000.0,
 'A_EX_slnt_e': 1000.0,
 'A_EX_phpyr_e': 1000,
 'B_EX_pi_e': 1000.0,
 'B_EX_h_e': 1000.0,
 'B_EX_fe3_e': 1000.0,
 'B_EX_mn2_e': 1000.0,
 'B_EX_co2_e': 1000.0,
 'B_EX_tyr__L_e': 1000,
 'B_EX_fe2_e': 1000.0,
 'B_EX_glc__D_e': 10,
 'B_EX_zn2_e': 1000.0,
 'B_EX_mg2_e': 1000.0,
 'B_EX_ca2_e': 1000.0,
 'B_EX_ni2_e': 1000.0,
 'B_EX_cu2_e': 1000.0,
 'B_EX_cobalt2_e': 1000.0,
 'B_EX_sel_e': 1000.0,
 'B_EX_h2o_e': 1000.0,
 'B_EX_nh4_e': 1000.0,
 'B_EX_mobd_e': 1000.0,
 'B_EX_so

In [1094]:
for i in origin_model.medium:
    model.add_reaction(origin_model.reactions.get_by_id(i))
    ex_meta=i.split('EX_')[1]
    model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
model.medium

{'A_EX_pi_e': 1000.0,
 'A_EX_h_e': 1000.0,
 'A_EX_fe3_e': 1000.0,
 'A_EX_mn2_e': 1000.0,
 'A_EX_co2_e': 1000.0,
 'A_EX_fe2_e': 1000.0,
 'A_EX_xyl__D_e': 10,
 'A_EX_zn2_e': 1000.0,
 'A_EX_mg2_e': 1000.0,
 'A_EX_ca2_e': 1000.0,
 'A_EX_ni2_e': 1000.0,
 'A_EX_cu2_e': 1000.0,
 'A_EX_cobalt2_e': 1000.0,
 'A_EX_sel_e': 1000.0,
 'A_EX_h2o_e': 1000.0,
 'A_EX_nh4_e': 1000.0,
 'A_EX_mobd_e': 1000.0,
 'A_EX_so4_e': 1000.0,
 'A_EX_k_e': 1000.0,
 'A_EX_na1_e': 1000.0,
 'A_EX_o2_e': 1000.0,
 'A_EX_cl_e': 1000.0,
 'A_EX_tungs_e': 1000.0,
 'A_EX_slnt_e': 1000.0,
 'A_EX_phpyr_e': 1000,
 'B_EX_pi_e': 1000.0,
 'B_EX_h_e': 1000.0,
 'B_EX_fe3_e': 1000.0,
 'B_EX_mn2_e': 1000.0,
 'B_EX_co2_e': 1000.0,
 'B_EX_tyr__L_e': 1000,
 'B_EX_fe2_e': 1000.0,
 'B_EX_glc__D_e': 10,
 'B_EX_zn2_e': 1000.0,
 'B_EX_mg2_e': 1000.0,
 'B_EX_ca2_e': 1000.0,
 'B_EX_ni2_e': 1000.0,
 'B_EX_cu2_e': 1000.0,
 'B_EX_cobalt2_e': 1000.0,
 'B_EX_sel_e': 1000.0,
 'B_EX_h2o_e': 1000.0,
 'B_EX_nh4_e': 1000.0,
 'B_EX_mobd_e': 1000.0,
 'B_EX_so

In [1095]:
for i in model.medium:
    h=i.split('_')[0]
    if h=='A' or h=='B':
        model.reactions.get_by_id(i).bounds=(0,0)
model.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_xyl__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [1096]:
model.add_reaction(model_b.reactions.EX_Salid_e)

In [1097]:
model_ab_json='./new_idea_AGGD/model/iML1515_AGGD_26.json'
save_json_model(model,model_ab_json)

In [104]:
model_ab_json='./new_idea_AGGD/model/iML1515_AGGD.json'
save_json_model(model,model_ab_json)

### 模型基本测试

In [ ]:
model_ab=load_json_model(model_ab_json)

In [ ]:
model_ab.reactions.	B_Salid_e_con_2

In [ ]:
model_ab.metabolites.B_Salid_e

In [ ]:
model_ab.metabolites.get_by_id('A_Tyrosol_e')

In [ ]:
model_ab.metabolites.get_by_id('B_Tyrosol_e')

In [ ]:
model_ab.reactions.get_by_id('A_mobd_e_con_1')

In [ ]:
model_ab.reactions.get_by_id('B_mobd_e_con_2')

In [ ]:
model_ab.reactions.get_by_id('EX_glc__D_e')

### change_model

### 代谢路径单向传输

In [1098]:
model_ab_json

'./new_idea_AGGD/model/iML1515_AGGD_26.json'

In [1099]:
model=load_json_model(model_ab_json)
model.reactions.get_by_id('B_Tyrosol_e_con_2').bounds=(0,1000)
model.reactions.get_by_id('A_Tyrosol_e_con_1').bounds=(0,1000)
model.reactions.get_by_id('B_tyr__L_e_con_2').bounds=(0,1000)
model.reactions.get_by_id('A_tyr__L_e_con_1').bounds=(0,1000)
model.reactions.get_by_id('B_phpyr_e_con_2').bounds=(-1000,0)
model.reactions.get_by_id('A_phpyr_e_con_1').bounds=(-1000,0)

In [1100]:
model.reactions.get_by_id('A_BIOMASS_Ec_iML1515_core_75p37M').bounds=(0.1,1000)
model.reactions.get_by_id('B_BIOMASS_Ec_iML1515_core_75p37M').bounds=(0.1,1000)

In [1101]:
model_ab_json_new='./new_idea_AGGD/model/iML1515_AGGD_26_change.json'

In [108]:
model_ab_json_new='./new_idea_AGGD/model/iML1515_AGGD_change.json'

In [1102]:
save_json_model(model,model_ab_json_new)

### 改变A均底物利用

In [110]:
model=load_json_model(model_ab_json_new)

In [111]:
model.reactions.A_glc__D_e_con_1.bounds=(0,1000)

In [112]:
save_json_model(model,model_ab_json_new)

### 后续预处理模型

In [ ]:
model_ab=load_json_model(model_ab_json_new)
cobra.manipulation.modify.convert_to_irreversible(model_ab)
isoenzyme_split(model_ab)

### 底物利用基于基因修改

In [ ]:
model_ab=load_json_model(model_ab_json_new)

In [ ]:
model_ab.reactions.A_glc__D_e_con_1.bounds=(-1000,1000)

In [ ]:
model_ab.reactions.get_by_id('A_ACMANAptspp').bounds=(0,0)

In [ ]:
model_ab.reactions.get_by_id('A_MANptspp').bounds=(0,0)

In [ ]:
model_ab.reactions.get_by_id('A_FRUpts2pp').bounds=(0,0)

In [ ]:
model_ab.reactions.get_by_id('A_GAMptspp').bounds=(0,0)

In [ ]:
model_ab.reactions.get_by_id('A_2DGLCptspp').bounds=(0,0)

In [ ]:
#xylA XYLT2
#print(model_ab.reactions.get_by_id('B_XYLI2'))
model_ab.reactions.get_by_id('B_XYLI2').bounds=(0,0)

In [ ]:
model_ab.reactions.get_by_id('A_HEX1').bounds=(0,0)

In [ ]:
model_ab.reactions.get_by_id('A_GLCptspp')

In [ ]:
new_change='./new_idea_AGGD/model/iML1515_AGGD_AG_glc_change.json'

In [ ]:
save_json_model(model_ab,new_change)

In [ ]:
model_test=load_json_model(new_change)

In [ ]:
model_test.reactions.A_glc__D_e_con_1.bounds=(0,1000)

In [ ]:
save_json_model(model_test,new_change)

### change xyl

In [ ]:
model_test.reactions.B_phpyr_e_con_2

In [ ]:
model_ab=load_json_model(new_change)

In [ ]:
for i in model_ab.reactions:
    if i.id.startswith('B_'):
        gen_list=[]
        for j in list(model_ab.reactions.get_by_id(i.id)._genes):
            gen_list.append(j.id)
        if len(gen_list)==1:
            if gen_list[0]=='b0480':
                model_ab.reactions.get_by_id(i.id).bounds=(0,0)

In [ ]:
save_json_model(model_ab,new_change)

In [ ]:
model_ab.reactions.A_glc__D_e_con_1

### close TYRTA

In [ ]:
model_ab=load_json_model(new_change)

In [ ]:
model_ab.reactions.B_TYRTA.bounds=(0,1000)

In [ ]:
save_json_model(model_ab,new_change)

### 限定生长(由于模型改造生长不受影响，复合最优生长+最高合成构建模型)

In [ ]:
model_ab=load_json_model(new_change)

In [ ]:
model_ab.reactions.B_BIOMASS_Ec_iML1515_core_75p37M.bounds=(0.1,1000)

In [ ]:
model_ab.reactions.A_BIOMASS_Ec_iML1515_core_75p37M.bounds=(0.1,1000)

In [ ]:
save_json_model(model_ab,new_change)

### change connect

In [1141]:
model_ab_json_new

'./new_idea_AGGD/model/iML1515_AGGD_change.json'

### 改变A菌只吸收木糖

In [35]:
model_change=load_json_model(model_ab_json_new)

In [36]:
model_change.reactions.get_by_id('B_BIOMASS_Ec_iML1515_core_75p37M').bounds=(0,1000)
model_change.reactions.get_by_id('A_BIOMASS_Ec_iML1515_core_75p37M').bounds=(0,1000)

In [37]:
model_change.reactions.A_glc__D_e_con_1.bounds=(-1000,1000)
model_change.reactions.B_xyl__D_e_con_2.bounds=(-1000,1000)

In [38]:
save_json_model(model_change,model_ab_json_new)

In [ ]:
# model_change=load_json_model(model_ab_json_new)
# # with model as model_change:
# reaction_list=model_change.reactions
# for m in reaction_list:
#     k=m.id
#     if k.endswith('_con_1') or k.endswith('_con_2'):
#         if 'glc__D_e_con' in k or 'xyl__D_e_con' in k or '_tyr__L_e_con' in k or 'Tyrosol_e' in k or 'phpyr_e' in k:
#             continue
#         model_change.reactions.get_by_id(k).bounds=(0,0)
#         #model_change.reactions.get_by_id(k+'_reverse').bounds=(0,0)
#         #print(k)

In [ ]:
# EX_change_a=pd.read_csv('./new_idea_AGGD/EC/EC_change_A.csv',index_col=0)
# EX_change_b=pd.read_csv('./new_idea_AGGD/EC/EC_change_B.csv',index_col=0)

In [ ]:
# for i in EX_change_a.index:
#     meta=i.split('EX_')[1]
#     if '_reverse' in meta:
#         me=meta.split('_reverse')[0]
#     else:
#         me=meta
#         model_change.reactions.get_by_id('A_'+meta+'_con_1').bounds=(0,1000)
#         model_change.reactions.get_by_id('A_'+meta+'_con_1_reverse').bounds=(0,1000)
#         model_change.reactions.get_by_id('B_'+meta+'_con_2').bounds=(0,1000)
#         model_change.reactions.get_by_id('B_'+meta+'_con_2_reverse').bounds=(0,1000)
# for i in EX_change_b.index:
#     meta=i.split('EX_')[1]
#     if '_reverse' in meta:
#         me=meta.split('_reverse')[0]
#     else:
#         me=meta
#         model_change.reactions.get_by_id('A_'+meta+'_con_1').bounds=(0,1000)
#         model_change.reactions.get_by_id('A_'+meta+'_con_1_reverse').bounds=(0,1000)
#         model_change.reactions.get_by_id('B_'+meta+'_con_2').bounds=(0,1000)
#         model_change.reactions.get_by_id('B_'+meta+'_con_2_reverse').bounds=(0,1000)

In [ ]:
# for i in EX_change_a.index:
#     meta=i.split('EX_')[1]
#     if '_reverse' in meta:
#         meta=meta.split('_reverse')[0]
#     model_change.reactions.get_by_id('A_'+meta+'_con_1').bounds=(-1000,1000)
#     model_change.reactions.get_by_id('B_'+meta+'_con_2').bounds=(-1000,1000)
# for i in EX_change_b.index:
#     meta=i.split('EX_')[1]
#     if '_reverse' in meta:
#         meta=meta.split('_reverse')[0]
#     model_change.reactions.get_by_id('A_'+meta+'_con_1').bounds=(-1000,1000)
#     model_change.reactions.get_by_id('B_'+meta+'_con_2').bounds=(-1000,1000)

In [ ]:
# model_change.reactions.get_by_id('B_Tyrosol_e_con_2').bounds=(0,1000)
# model_change.reactions.get_by_id('A_Tyrosol_e_con_1').bounds=(0,1000)
# model_change.reactions.get_by_id('B_tyr__L_e_con_2').bounds=(0,1000)
# model_change.reactions.get_by_id('A_tyr__L_e_con_1').bounds=(0,1000)
# model_change.reactions.get_by_id('B_phpyr_e_con_2').bounds=(-1000,0)
# model_change.reactions.get_by_id('A_phpyr_e_con_1').bounds=(-1000,0)

In [ ]:
# save_json_model(model_change,model_ab_json_new)

### new_idea

In [6]:
from new_cobra import *

In [7]:
def Get_Model_Data(model):
    """Returns reaction_list,metabolite_list,lb_list,ub_list,coef_matrix from model.
    
    Notes: 
    ----------
    *model： is in SBML format (.xml).
    """
    reaction_list=[]
    reaction_list_A=[]
    reaction_list_B=[]
    metabolite_list=[]
    lb_list={}
    ub_list={}
    coef_matrix={}
    for rea in model.reactions:
        reaction_list.append(rea.id)
        if rea.id.startswith('A_'):
            reaction_list_A.append(rea.id)
        if rea.id.startswith('B_'):
            reaction_list_B.append(rea.id)    
        lb_list[rea.id]=rea.lower_bound
        ub_list[rea.id]=rea.upper_bound
        for met in model.metabolites:
            metabolite_list.append(met.id)
            try:
                rea.get_coefficient(met.id)  
            except:
                pass
            else:
                coef_matrix[met.id,rea.id]=rea.get_coefficient(met.id)
    reaction_list=list(set(reaction_list))
    metabolite_list=list(set(metabolite_list))
    return(reaction_list,reaction_list_A,reaction_list_B,metabolite_list,lb_list,ub_list,coef_matrix)

In [8]:
def Get_Concretemodel_Need_Data_json(model_file,reaction_kcat_MW_file,target):
    Concretemodel_Need_Data={}
    # reaction_g0=pd.read_csv(reaction_g0_file,index_col=0,sep='\t')
    # Concretemodel_Need_Data['reaction_g0']=reaction_g0
    # metabolites_lnC = pd.read_csv(metabolites_lnC_file, index_col=0, sep='\t')
    # Concretemodel_Need_Data['metabolites_lnC']=metabolites_lnC
    model=cobra.io.load_json_model(model_file)
    try:
        product = model.metabolites.get_by_id(target)
        model.add_boundary(product, type='demand')  # add demand reaction as the objective
    except:
        pass
    cobra.manipulation.modify.convert_to_irreversible(model)
    isoenzyme_split(model)
    reaction_kcat_MW=pd.read_csv(reaction_kcat_MW_file,index_col=0)
    Concretemodel_Need_Data['model']=model
    Concretemodel_Need_Data['reaction_kcat_MW']=reaction_kcat_MW
    [reaction_list,reaction_list_A,reaction_list_B,metabolite_list,lb_list,ub_list,coef_matrix]=Get_Model_Data(model)
    Concretemodel_Need_Data['reaction_list']=reaction_list
    Concretemodel_Need_Data['reaction_list_A']=reaction_list_A
    Concretemodel_Need_Data['reaction_list_B']=reaction_list_B
    Concretemodel_Need_Data['metabolite_list']=metabolite_list
    Concretemodel_Need_Data['lb_list']=lb_list
    Concretemodel_Need_Data['ub_list']=ub_list
    Concretemodel_Need_Data['coef_matrix']=coef_matrix
    return (Concretemodel_Need_Data)

#### model初始化

In [21]:
model_ab_json_new='./new_idea_AGGD/model/iML1515_AGGD_change.json'

In [66]:
# model_file=model_ab_json
#model_file=new_change
model_file=model_ab_json_new
reaction_kcat_MW_file='./new_idea_AGGD/EC/reaction_kcat_MW.csv'
Concretemodel_Need_Data=Get_Concretemodel_Need_Data_json(model_file,reaction_kcat_MW_file,'Salid_c')

In [67]:
Concretemodel_Need_Data['reaction_kcat_MW']=unpdate_kcat_mw(Concretemodel_Need_Data['reaction_list'],Concretemodel_Need_Data['reaction_kcat_MW'],'./new_idea_AGGD/EC/reaction_kcat_MW_AGGD.csv')

In [65]:
def Template_Concretemodel2(reaction_list=None,metabolite_list=None,coef_matrix=None,reaction_kcat_MW=None,lb_list=None,ub_list=None,reaction_list_A=None,reaction_list_B=None,\
    set_substrate_ini=False,substrate_name=None,substrate_value=None,substrates=None,substrates_bool=False,\
    set_biomass_ini=False,biomass_value=None,biomass_list=None,biomass_id=None,set_A_biomass_ini=None,set_B_biomass_ini=None,\
    set_bound=False,set_stoi_matrix=False,set_enzyme_constraint=False,set_part_enzyme_constraint=False,set_obj_single_E_value=False,E_total=None,\
    obj_name=None,obj_target=None,set_obj_value=False,set_target_ini=False,target_value=None,\
    set_obj_V_value=False,set_new_stoi_matrix=False,set_obj_E_value=False,\
    set_ratio=False,set_ratio_bound=False,set_pass_ini=False,set_pass_value=None,\
    set_metabolite=False,set_obj_Met_value=False,metabolites_lnC=None,\
    set_sub_ratio=False,sub_a=None,sub_b=None,sub_ratio=None):
    

    Concretemodel = ConcreteModel()
    Concretemodel.reaction = pyo.Var(reaction_list,  within=NonNegativeReals)
    Concretemodel.z = pyo.Var(reaction_list,  within=pyo.Binary)
    Concretemodel.x = pyo.Var()
    Concretemodel.y = pyo.Var()

    #Set upper and lower bounds of metabolite concentration
    # if set_metabolite:
    #     def set_metabolite(m,i):
    #         return  inequality(metabolites_lnC.loc[i,'lnClb'], m.metabolite[i], metabolites_lnC.loc[i,'lnCub'])
    #     Concretemodel.set_metabolite= Constraint(metabolite_list,rule=set_metabolite)  

    if set_ratio:
        def set_ratio(m):
            return  m.x+m.y==100
        Concretemodel.set_ratio= Constraint(rule=set_ratio)    
        
    if set_ratio_bound:
        def set_ratio_bound(m):
            return inequality(1,m.x,99)
        Concretemodel.set_ratio_bound = Constraint(rule=set_ratio_bound)     

    #Minimizing the flux sum of pathway (pFBA)
    if set_obj_V_value:             
        def set_obj_V_value(m):
            return sum(m.reaction[j] for j in reaction_list)
        Concretemodel.obj = Objective(rule=set_obj_V_value, sense=minimize)  
    
    #Set the maximum flux as the object function
    if set_obj_value:   
        def set_obj_value(m):
            return m.reaction[obj_name]
        if obj_target=='maximize':
            Concretemodel.obj = Objective(rule=set_obj_value, sense=maximize)
        elif obj_target=='minimize':
            Concretemodel.obj = Objective(rule=set_obj_value, sense=minimize)

    #Set the minimum enzyme cost of a pathway as the object function
    if set_obj_E_value:             
        def set_obj_E_value(m):
            return sum(m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index)
        Concretemodel.obj = Objective(rule=set_obj_E_value, sense=minimize)  

    #To calculate the variability of enzyme usage of single reaction.
    if set_obj_single_E_value:             
        def set_obj_single_E_value(m):
            return m.reaction[obj_name]/(reaction_kcat_MW.loc[obj_name,'kcat_MW'])
        if obj_target=='maximize':
            Concretemodel.obj = Objective(rule=set_obj_single_E_value, sense=maximize) 
        elif obj_target=='minimize':
            Concretemodel.obj = Objective(rule=set_obj_single_E_value, sense=minimize)

    if set_sub_ratio:
        def set_sub_ratio(m):
            return m.reaction[sub_a]==4*m.reaction[sub_b]
        Concretemodel.set_sub_ratio = Constraint(rule=set_sub_ratio)

    #To calculate the concentration variability of metabolites.
    # if set_obj_Met_value:   
    #     def set_obj_Met_value(m):
    #         return m.metabolite[obj_name]
    #     if obj_target=='maximize':
    #         Concretemodel.obj = Objective(rule=set_obj_Met_value, sense=maximize)
    #     elif obj_target=='minimize':
    #         Concretemodel.obj = Objective(rule=set_obj_Met_value, sense=minimize)

    #Adding flux balance constraints （FBA）
    if set_stoi_matrix:
        def set_stoi_matrix(m,i):
            return sum(coef_matrix[i,j]*m.reaction[j]  for j in reaction_list if (i,j) in coef_matrix.keys() )==0
        Concretemodel.set_stoi_matrix = Constraint( metabolite_list,rule=set_stoi_matrix)

    #Adding flux balance constraints （newFBA）
    if set_new_stoi_matrix:
        def set_new_stoi_matrix(m,i):
            for j in reaction_list:
                if (i,j) in coef_matrix.keys():
                    if j.startswith('A_') and i.startswith('A_') and '_con_' in j:
                        coef_matrix[i,j]=(1/m.x)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                                # print ('A:'+str(coef_matrix[i,j]))
                    if j.startswith('B_') and i.startswith('B_') and '_con_' in j:
                        coef_matrix[i,j]=(1/m.y)*coef_matrix[i,j]
            return sum(coef_matrix[i,j]*m.reaction[j]  for j in reaction_list if (i,j) in coef_matrix.keys() )==0
        Concretemodel.set_new_stoi_matrix = Constraint( metabolite_list,rule=set_new_stoi_matrix)

    #Adding the upper and lower bound constraints of reaction flux
    if set_bound:
        def set_bound(m,j):
            return inequality(lb_list[j],m.reaction[j],ub_list[j])
        Concretemodel.set_bound = Constraint(reaction_list,rule=set_bound) 

    #Set the upper bound for substrate input reaction flux
    if set_substrate_ini and substrates_bool==False:
        def set_substrate_ini(m): 
            return m.reaction[substrate_name] <= substrate_value
        Concretemodel.set_substrate_ini = Constraint(rule=set_substrate_ini)  
    
     #Set the upper bound for substrate input reaction flux
    if set_substrate_ini and substrates_bool==True:
        def set_substrates_ini(m,j): 
            return m.reaction[j]<= substrates[j]
        Concretemodel.set_substrates_ini = Constraint(substrates.keys(),rule=set_substrates_ini)  
    
    if set_biomass_ini:
        def set_biomass_ini(m): 
            return m.reaction[biomass_id] >=biomass_value
        Concretemodel.set_biomass_ini = Constraint(rule=set_biomass_ini)  
        
    #Set the lower bound for biomass synthesis reaction flux
    if set_A_biomass_ini:
        def set_A_biomass_ini(m): 
            return m.reaction['A_'+biomass_id] >=biomass_list['A']
        Concretemodel.set_A_biomass_ini = Constraint(rule=set_A_biomass_ini)  
        
    #Set the lower bound for biomass synthesis reaction flux
    if set_B_biomass_ini:
        def set_B_biomass_ini(m): 
            return m.reaction['B_'+biomass_id] >=biomass_list['B']
        Concretemodel.set_B_biomass_ini = Constraint(rule=set_B_biomass_ini)  
    
    #Set the lower bound for target synthesis reaction flux
    if set_target_ini:
        def set_target_ini(m): 
            return m.reaction[obj_name] ==target_value
        Concretemodel.set_target_ini = Constraint(rule=set_target_ini)  

    #Adding enzymamic constraints
    if set_enzyme_constraint:
        def set_enzyme_constraint(m):
            return sum( m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index if j in reaction_list)<= E_total
        Concretemodel.set_enzyme_constraint = Constraint(rule=set_enzyme_constraint)    
    
    #Adding Part A enzymamic constraints
    if set_part_enzyme_constraint:
        def set_enzyme_constraint_A(m):
            return sum( m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index if j in reaction_list_A)<= E_total
        Concretemodel.set_enzyme_constraint_A = Constraint(rule=set_enzyme_constraint_A)

    #Adding Part B enzymamic constraints
    if set_part_enzyme_constraint:
        def set_enzyme_constraint_B(m):
            return sum( m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index if j in reaction_list_B)<= E_total
        Concretemodel.set_enzyme_constraint_B = Constraint(rule=set_enzyme_constraint_B)

    #Adding pass meta flux constraints
    if set_pass_ini:
        def set_pass_ini(m,j):
            return m.reaction[j] >= set_pass_value[j]
        Concretemodel.set_pass_ini = Constraint(set_pass_value.keys(),rule=set_pass_ini)  

    return Concretemodel

In [12]:
def Model_Solve(model,solver):
    opt = pyo.SolverFactory(solver)
    opt.solve(model)
    return model

In [ ]:
# Concretemodel_Need_Data['reaction_list']

In [ ]:
# Concretemodel_Need_Data['coef_matrix']

### FBA

In [13]:
def ECM_FBA(a,b,x,y,mid_a,mid_b,bio_a,bio_b):
    #EcoECM=EcoECM(Concretemodel_Need_Data,'DM_B_C06104','maximize',substrates,E_total)\
    #substrates={'EX_glc__D_e_reverse':10}
    substrates={'EX_glc__D_e_reverse':a,'EX_xyl__D_e_reverse':b}
    substrate_value=10
    substrate_name='EX_glc__D_e_reverse'
    E_total=0.227
    sub_a='EX_glc__D_e_reverse'
    sub_b='EX_xyl__D_e_reverse'
    ratio=4
    reaction_list_A=deepcopy(Concretemodel_Need_Data['reaction_list_A'])
    reaction_list_B=deepcopy(Concretemodel_Need_Data['reaction_list_B'])
    reaction_list=deepcopy(Concretemodel_Need_Data['reaction_list'])
    metabolite_list=deepcopy(Concretemodel_Need_Data['metabolite_list'])
    coef_matrix=deepcopy(Concretemodel_Need_Data['coef_matrix'])
    for i in metabolite_list:
        for j in reaction_list:
                if (i,j) in coef_matrix.keys():
                    if j.startswith('A_') and i.startswith('A_') and '_con_' in j:
                        coef_matrix[i,j]=(1/x)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                                # print ('A:'+str(coef_matrix[i,j]))
                    if j.startswith('B_') and i.startswith('B_') and '_con_' in j:
                        coef_matrix[i,j]=(1/y)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                #                 print ('B:'+str(coef_matrix[i,j]))
                #     print ('ij:'+str(coef_matrix[i,j]))
    reaction_kcat_MW=deepcopy(Concretemodel_Need_Data['reaction_kcat_MW'])
    lb_list=Concretemodel_Need_Data['lb_list']
    ub_list=Concretemodel_Need_Data['ub_list']
    if len(substrates.keys())==1:
            for i in substrates.keys():
                ub_list[i]=substrates[i]
                substrate_name=i
                substrate_value=substrates[i]
            substrates_bool=False
    else:
            for i in substrates.keys():
                    ub_list[i]=substrates[i]
            substrate_name=''
            substrate_value=0
            substrates_bool=True

    obj_name='EX_Salid_e'
    obj_target='maximize'
    biomass_id='BIOMASS_Ec_iML1515_core_75p37M'
    pass_value={'A_tyr__L_e_con_1':mid_a,'B_phpyr_e_con_2_reverse':mid_b}
    EcoECM_FBA=Template_Concretemodel2(reaction_list=reaction_list,metabolite_list=metabolite_list,coef_matrix=coef_matrix,\
        reaction_list_A=reaction_list_A,reaction_list_B=reaction_list_B,reaction_kcat_MW=reaction_kcat_MW,lb_list=lb_list,ub_list=ub_list,\
        obj_name=obj_name,obj_target=obj_target,set_obj_value=True,biomass_id=biomass_id,set_substrate_ini=True,substrate_name=substrate_name,\
        substrate_value=substrate_value,substrates=substrates,substrates_bool=substrates_bool,set_stoi_matrix=True,set_bound=True,E_total=E_total,\
        set_enzyme_constraint=False,set_part_enzyme_constraint=True,set_A_biomass_ini=True,set_B_biomass_ini=True,biomass_list={'A':bio_a,'B':bio_b},\
        set_pass_ini=True,set_pass_value=pass_value,set_sub_ratio=False,sub_a=sub_a,sub_b=sub_b,sub_ratio=ratio)#{'A':0.15,'B':0.65}
    opt_ecm_FBA=Model_Solve(EcoECM_FBA,'gurobi')
    #opt_ecm_pFBA.obj()
    return opt_ecm_FBA

### pFBA

In [14]:
def ECM_pFBA(a,b,x,y,obj,mid_a,mid_b,bio_a,bio_b):
    #EcoECM=EcoECM(Concretemodel_Need_Data,'DM_B_C06104','maximize',substrates,E_total)\
    #substrates={'EX_glc__D_e_reverse':10}
    substrates={'EX_glc__D_e_reverse':a,'EX_xyl__D_e_reverse':b}
    substrate_value=10
    substrate_name=''
    E_total=0.227
    sub_a='EX_glc__D_e_reverse'
    sub_b='EX_xyl__D_e_reverse'
    ratio=4
    reaction_list_A=deepcopy(Concretemodel_Need_Data['reaction_list_A'])
    reaction_list_B=deepcopy(Concretemodel_Need_Data['reaction_list_B'])
    reaction_list=deepcopy(Concretemodel_Need_Data['reaction_list'])
    metabolite_list=deepcopy(Concretemodel_Need_Data['metabolite_list'])
    coef_matrix=deepcopy(Concretemodel_Need_Data['coef_matrix'])
    for i in metabolite_list:
        for j in reaction_list:
                if (i,j) in coef_matrix.keys():
                    if j.startswith('A_') and i.startswith('A_') and '_con_' in j:
                        coef_matrix[i,j]=(1/x)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                                # print ('A:'+str(coef_matrix[i,j]))
                    if j.startswith('B_') and i.startswith('B_') and '_con_' in j:
                        coef_matrix[i,j]=(1/y)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                #                 print ('B:'+str(coef_matrix[i,j]))
                #     print ('ij:'+str(coef_matrix[i,j]))
    reaction_kcat_MW=deepcopy(Concretemodel_Need_Data['reaction_kcat_MW'])
    lb_list=Concretemodel_Need_Data['lb_list']
    ub_list=Concretemodel_Need_Data['ub_list']
    # if len(substrates.keys())==1:
    #         for i in substrates.keys():
    #             ub_list[i]=substrates[i]
    #             substrate_name=i
    #             substrate_value=substrates[i]
    #         substrates_bool=False
    # else:
    #         for i in substrates.keys():
    #                 ub_list[i]=substrates[i]
    #         substrate_name=''
    #         substrate_value=0
    #         substrates_bool=True
    substrates_bool=True
    obj_name='EX_Salid_e'
    obj_target='maximize'
    biomass_id='BIOMASS_Ec_iML1515_core_75p37M'
    #pass_value={'A_tyr__L_e_con_1':mid_a,'B_phpyr_e_con_2_reverse':mid_b}
    EcoECM_pFBA=Template_Concretemodel2(reaction_list=reaction_list,metabolite_list=metabolite_list,coef_matrix=coef_matrix,\
        reaction_list_A=reaction_list_A,reaction_list_B=reaction_list_B,reaction_kcat_MW=reaction_kcat_MW,lb_list=lb_list,ub_list=ub_list,\
        obj_name=obj_name,obj_target=obj_target,set_obj_value=False,set_target_ini=True,set_obj_V_value=True,target_value=obj,\
        biomass_id=biomass_id,set_A_biomass_ini=True,set_B_biomass_ini=True,biomass_list={'A':bio_a,'B':bio_b},set_substrate_ini=True,substrate_name=substrate_name,\
        substrate_value=substrate_value,substrates=substrates,substrates_bool=substrates_bool,set_stoi_matrix=True,set_bound=True,E_total=E_total,\
        set_enzyme_constraint=False,set_part_enzyme_constraint=True,set_pass_ini=False,set_pass_value=None,set_sub_ratio=False,sub_a=sub_a,sub_b=sub_b,sub_ratio=ratio)#{'A':0.1,'B':0.65}
    opt_ecm_pFBA=Model_Solve(EcoECM_pFBA,'gurobi')
    #opt_ecm_pFBA.obj()
    return opt_ecm_pFBA

### test_new

In [ ]:
# def test(a):
#     #EcoECM=EcoECM(Concretemodel_Need_Data,'DM_B_C06104','maximize',substrates,E_total)\
#     #substrates={'EX_glc__D_e_reverse':10}
#     substrates={'EX_glc__D_e_reverse':a,'EX_xyl__D_e_reverse':200}
#     substrate_value=10
#     substrate_name='EX_glc__D_e_reverse'
#     E_total=0.227
#     reaction_list_A=deepcopy(Concretemodel_Need_Data['reaction_list_A'])
#     reaction_list_B=deepcopy(Concretemodel_Need_Data['reaction_list_B'])
#     reaction_list=deepcopy(Concretemodel_Need_Data['reaction_list'])
#     metabolite_list=deepcopy(Concretemodel_Need_Data['metabolite_list'])
#     coef_matrix=deepcopy(Concretemodel_Need_Data['coef_matrix'])
#     # for i in metabolite_list:
#     #     for j in reaction_list:
#     #             if (i,j) in coef_matrix.keys():
#     #                 if j.startswith('A_') and i.startswith('A_') and '_con_' in j:
#     #                     coef_matrix[i,j]=(1/46)*coef_matrix[i,j]
#     #                     # if 'akg_e_con_' in j:
#     #                             # print ('A:'+str(coef_matrix[i,j]))
#     #                 if j.startswith('B_') and i.startswith('B_') and '_con_' in j:
#     #                     coef_matrix[i,j]=(1/64)*coef_matrix[i,j]
#     #                     # if 'akg_e_con_' in j:
#     #             #                 print ('B:'+str(coef_matrix[i,j]))
#     #             #     print ('ij:'+str(coef_matrix[i,j]))
#     reaction_kcat_MW=deepcopy(Concretemodel_Need_Data['reaction_kcat_MW'])
#     lb_list=Concretemodel_Need_Data['lb_list']
#     ub_list=Concretemodel_Need_Data['ub_list']
#     if len(substrates.keys())==1:
#             for i in substrates.keys():
#                     ub_list[i]=substrates[i]
#             substrate_name=i
#             substrate_value=substrates[i]
#             substrates_bool=False
#     else:
#             for i in substrates.keys():
#                     ub_list[i]=substrates[i]
#             substrate_name=''
#             substrate_value=0
#             substrates_bool=True

#     obj_name='B_DM_C06104'
#     obj_target='maximize'
#     biomass_id='BIOMASS_Ec_iML1515_core_75p37M'
#     EcoECM_FBA=Template_Concretemodel2(reaction_list=reaction_list,metabolite_list=metabolite_list,coef_matrix=coef_matrix,reaction_list_A=reaction_list_A,reaction_list_B=reaction_list_B,\
#         reaction_kcat_MW=reaction_kcat_MW,lb_list=lb_list,ub_list=ub_list,obj_name=obj_name,obj_target=obj_target,set_obj_value=True,biomass_id=biomass_id,set_ratio=True,set_ratio_bound=True,\
#         set_substrate_ini=True,substrate_name=substrate_name,substrate_value=substrate_value,substrates=substrates,substrates_bool=substrates_bool,set_new_stoi_matrix=True,set_bound=True,\
#         set_enzyme_constraint=False,set_part_enzyme_constraint=True,E_total=E_total,set_A_biomass_ini=False,set_B_biomass_ini=False,biomass_value=None)
#     opt_ecm_FBA=Model_Solve(EcoECM_FBA,'gurobi')
#     #opt_ecm_pFBA.obj()
#     return opt_ecm_FBA

In [ ]:
# d=test(200)
# d.obj()
# d.x()

In [ ]:
# def ECM_pFBA(a,obj):
#     #EcoECM=EcoECM(Concretemodel_Need_Data,'DM_B_C06104','maximize',substrates,E_total)\
#     #substrates={'EX_glc__D_e_reverse':10}
#     substrates={'EX_glc__D_e_reverse':a,'EX_xyl__D_e_reverse':200}
#     substrate_value=10
#     substrate_name='EX_glc__D_e_reverse'
#     E_total=0.227
#     reaction_list_A=deepcopy(Concretemodel_Need_Data['reaction_list_A'])
#     reaction_list_B=deepcopy(Concretemodel_Need_Data['reaction_list_B'])
#     reaction_list=deepcopy(Concretemodel_Need_Data['reaction_list'])
#     metabolite_list=deepcopy(Concretemodel_Need_Data['metabolite_list'])
#     coef_matrix=deepcopy(Concretemodel_Need_Data['coef_matrix'])
#     for i in metabolite_list:
#         for j in reaction_list:
#                 if (i,j) in coef_matrix.keys():
#                     if j.startswith('A_') and i.startswith('A_') and '_con_' in j:
#                         coef_matrix[i,j]=(1/46)*coef_matrix[i,j]
#                         # if 'akg_e_con_' in j:
#                                 # print ('A:'+str(coef_matrix[i,j]))
#                     if j.startswith('B_') and i.startswith('B_') and '_con_' in j:
#                         coef_matrix[i,j]=(1/64)*coef_matrix[i,j]
#                         # if 'akg_e_con_' in j:
#                 #                 print ('B:'+str(coef_matrix[i,j]))
#                 #     print ('ij:'+str(coef_matrix[i,j]))
#     reaction_kcat_MW=deepcopy(Concretemodel_Need_Data['reaction_kcat_MW'])
#     lb_list=Concretemodel_Need_Data['lb_list']
#     ub_list=Concretemodel_Need_Data['ub_list']
#     if len(substrates.keys())==1:
#             for i in substrates.keys():
#                     ub_list[i]=substrates[i]
#             substrate_name=i
#             substrate_value=substrates[i]
#             substrates_bool=False
#     else:
#             for i in substrates.keys():
#                     ub_list[i]=substrates[i]
#             substrate_name=''
#             substrate_value=0
#             substrates_bool=True

#     obj_name='B_DM_C06104'
#     obj_target='maximize'
#     biomass_id='BIOMASS_Ec_iML1515_core_75p37M'
#     EcoECM_pFBA=Template_Concretemodel(reaction_list=reaction_list,metabolite_list=metabolite_list,coef_matrix=coef_matrix,reaction_list_A=reaction_list_A,reaction_list_B=reaction_list_B,\
#     reaction_kcat_MW=reaction_kcat_MW,lb_list=lb_list,ub_list=ub_list,obj_name=obj_name,obj_target=obj_target,set_obj_value=False,set_target_ini=True,target_id=obj_name,target_value=obj,\
#     set_substrate_ini=True,substrate_name=substrate_name,substrate_value=substrate_value,substrates=substrates,substrates_bool=substrates_bool,set_stoi_matrix=True,set_bound=True,set_obj_V_value=True,\
#     set_enzyme_constraint=False,set_part_enzyme_constraint=True,E_total=E_total)
#     opt_ecm_pFBA=Model_Solve(EcoECM_pFBA,'gurobi')
#     #opt_ecm_pFBA.obj()
#     return opt_ecm_pFBA

In [ ]:
# def ECM_pFBA2(a):
#     #EcoECM=EcoECM(Concretemodel_Need_Data,'DM_B_C06104','maximize',substrates,E_total)\
#     #substrates={'EX_glc__D_e_reverse':10}
#     substrates={'EX_glc__D_e_reverse':a,'EX_akg_e_reverse':0}
#     substrate_value=10
#     substrate_name='EX_glc__D_e_reverse'
#     E_total=0.227
#     reaction_list=Concretemodel_Need_Data['reaction_list']
#     metabolite_list=Concretemodel_Need_Data['metabolite_list']
#     coef_matrix=Concretemodel_Need_Data['coef_matrix']
#     reaction_kcat_MW=Concretemodel_Need_Data['reaction_kcat_MW']
#     lb_list=Concretemodel_Need_Data['lb_list']
#     ub_list=Concretemodel_Need_Data['ub_list']
#     if len(substrates.keys())==1:
#             for i in substrates.keys():
#                     ub_list[i]=substrates[i]
#             substrate_name=i
#             substrate_value=substrates[i]
#             substrates_bool=False
#     else:
#             for i in substrates.keys():
#                     ub_list[i]=substrates[i]
#             substrate_name=''
#             substrate_value=0
#             substrates_bool=True

#     obj_name='DM_akg_c'
#     obj_target='maximize'
#     biomass_id='BIOMASS_Ec_iML1515_core_75p37M'
#     EcoECM_pFBA=Template_Concretemodel(reaction_list=reaction_list,metabolite_list=metabolite_list,coef_matrix=coef_matrix,\
#         reaction_kcat_MW=reaction_kcat_MW,lb_list=lb_list,ub_list=ub_list,obj_name=obj_name,obj_target=obj_target,set_obj_value=True,biomass_id=biomass_id,\
#         set_substrate_ini=True,substrate_name=substrate_name,substrate_value=substrate_value,substrates=substrates,substrates_bool=substrates_bool,set_stoi_matrix=True,set_bound=True,\
#         set_enzyme_constraint=True,set_part_enzyme_constraint=False,E_total=E_total,biomass_value=None)
#     opt_ecm_pFBA=Model_Solve(EcoECM_pFBA,'gurobi')
#     #opt_ecm_pFBA.obj()
#     return opt_ecm_pFBA

### output

In [ ]:
result_ratio=pd.DataFrame()

In [ ]:
for m in range(1,100):
    n=100-m
    try:
        a=ECM_FBA(800,200,m,n)
        result_ratio.at[m,'S_value']=a.obj()*n
    except:
        print(str(m)+':error')
        continue
    # if (a.reaction['A_Tyrosol_e_con_1'].value-a.reaction['A_Tyrosol_e_con_1_reverse'].value)>=0 or (a.reaction['A_tyr__L_e_con_1'].value-a.reaction['A_tyr__L_e_con_1_reverse'].value)<=0\
    #      or (a.reaction['A_phpyr_e_con_1'].value-a.reaction['A_phpyr_e_con_1_reverse'].value)<=0:
    #     pass
    result_ratio.at[m,'A']=m
    result_ratio.at[m,'B']=n
    result_ratio.at[m,'obj']=a.obj()
    result_ratio.at[m,'S_value']=a.obj()*n
    result_ratio.at[m,'A_glc_out']=a.reaction['A_glc__D_e_con_1'].value
    result_ratio.at[m,'B_glc']=a.reaction['B_glc__D_e_con_2'].value-a.reaction['B_glc__D_e_con_2_reverse'].value
    result_ratio.at[m,'A_xyl']=a.reaction['A_xyl__D_e_con_1_reverse'].value-a.reaction['A_xyl__D_e_con_1'].value
    result_ratio.at[m,'B_xyl']=a.reaction['B_xyl__D_e_con_2'].value-a.reaction['B_xyl__D_e_con_2_reverse'].value
    result_ratio.at[m,'A_c2e']=a.reaction['A_Tyrosol_c2e'].value-a.reaction['A_Tyrosol_c2e_reverse'].value
    result_ratio.at[m,'T_A_value']=a.reaction['A_Tyrosol_e_con_1'].value
    result_ratio.at[m,'T_B_value']=a.reaction['B_Tyrosol_e_con_2'].value
    result_ratio.at[m,'B_e2c']=a.reaction['B_Tyrosol_c2e_reverse'].value-a.reaction['B_Tyrosol_c2e'].value
    print(str(a.obj()*n),m,result_ratio.at[m,'A_c2e'],result_ratio.at[m,'T_A_value'],result_ratio.at[m,'T_B_value'],result_ratio.at[m,'B_e2c'])

In [ ]:
result_ratio

In [ ]:
result_ratio.to_csv('./new_idea_AGGD/result/4_2;1.csv')

In [ ]:
### change_glc底物摄入
result_ratio.to_csv('./new_idea_AGGD/result/4_change_glc.csv')

In [ ]:
a=ECM_FBA(800,200,80,20)
a.obj()*20

In [ ]:
a.reaction['B_DM_Salid_c'].value

In [ ]:
a.obj()

In [ ]:
result_ratio_single=pd.DataFrame()
for i in Concretemodel_Need_Data['reaction_list']:
    try:
        result_ratio_single.at[i,'flux']=a.reaction[i].value
        result_ratio_single.at[i,'kcat_MW']=Concretemodel_Need_Data['reaction_kcat_MW'].at[i,'kcat_MW']
        result_ratio_single.at[i,'enz_useage']=round(result_ratio_single.at[i,'flux']/result_ratio_single.at[i,'kcat_MW'],6)
    except:
        pass
    result_ratio_single.at[i,'formula']=model_ab.reactions.get_by_id(i).reaction
result_ratio_single.to_csv('./new_idea_AGGD/result/1.csv')

In [ ]:
result_ratio_pfba=pd.DataFrame()

In [ ]:
# result_ratio.at[80,'S_value']

In [ ]:
# b=ECM_pFBA(800,200,80,20,a.obj())

In [ ]:
for m in range(1,100):
    n=100-m
    obj=result_ratio.at[m,'obj']
    print(obj)
    try:
        a=ECM_pFBA(800,200,m,n,obj)
        #=a.obj()*n
    except:
        print(str(m)+':error')
        continue
    # if (a.reaction['A_Tyrosol_e_con_1'].value-a.reaction['A_Tyrosol_e_con_1_reverse'].value)>=0 or (a.reaction['A_tyr__L_e_con_1'].value-a.reaction['A_tyr__L_e_con_1_reverse'].value)<=0\
    #      or (a.reaction['A_phpyr_e_con_1'].value-a.reaction['A_phpyr_e_con_1_reverse'].value)<=0:
    #     pass
    result_ratio_pfba.at[m,'A']=m
    result_ratio_pfba.at[m,'B']=n
    result_ratio_pfba.at[m,'obj']=obj
    result_ratio_pfba.at[m,'S_value']=a.reaction['B_DM_Salid_c'].value*n
    result_ratio_pfba.at[m,'A_glc']=a.reaction['A_glc__D_e_con_1_reverse'].value-a.reaction['A_glc__D_e_con_1'].value
    result_ratio_pfba.at[m,'B_glc']=a.reaction['B_glc__D_e_con_2'].value-a.reaction['B_glc__D_e_con_2_reverse'].value
    result_ratio_pfba.at[m,'A_xyl']=a.reaction['A_xyl__D_e_con_1_reverse'].value-a.reaction['A_xyl__D_e_con_1'].value
    result_ratio_pfba.at[m,'B_xyl']=a.reaction['B_xyl__D_e_con_2'].value-a.reaction['B_xyl__D_e_con_2_reverse'].value
    result_ratio_pfba.at[m,'A_c2e']=a.reaction['A_Tyrosol_c2e'].value-a.reaction['A_Tyrosol_c2e_reverse'].value
    result_ratio_pfba.at[m,'T_A_value']=a.reaction['A_Tyrosol_e_con_1'].value
    result_ratio_pfba.at[m,'T_B_value']=a.reaction['B_Tyrosol_e_con_2'].value
    result_ratio_pfba.at[m,'B_e2c']=a.reaction['B_Tyrosol_c2e_reverse'].value-a.reaction['B_Tyrosol_c2e'].value
    print(result_ratio_pfba.at[m,'S_value'],m,result_ratio_pfba.at[m,'A_c2e'],result_ratio_pfba.at[m,'T_A_value'],result_ratio_pfba.at[m,'T_B_value'],result_ratio_pfba.at[m,'B_e2c'])

In [ ]:
result_ratio_pfba.to_csv('./new_idea_AGGD/result/4_4;1_pfba.csv')

### test_ratio

In [15]:
def get_two_float(f_str, n):
    a, b, c = str(f_str).partition('.')
    c = c[:n]
    return float(".".join([a, c]))

## test_kcat

In [5]:
from double_ECM import *

In [6]:
list_x=[9,8,6,4]
list_y=[3,4,6,8]

In [45]:
# list_x=[3,2,1,1]
# list_y=[1,1,1,2]

In [7]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import datetime
from thread import *
import sys
sys.path.append(r'./')

In [8]:
model_file=model_ab_json_new
reaction_kcat_MW_file='./new_idea_AGGD/EC/reaction_kcat_MW_AGGD_origin.csv'
Concretemodel_Need_Data=Get_Concretemodel_Need_Data_json(model_file,reaction_kcat_MW_file,'Salid_c')
Concretemodel_Need_Data['reaction_kcat_MW']=unpdate_kcat_mw(Concretemodel_Need_Data['reaction_list'],Concretemodel_Need_Data['reaction_kcat_MW'],'./new_idea_AGGD/EC/reaction_kcat_MW_AGGD.csv')

In [445]:
Concretemodel_Need_Data['reaction_kcat_MW']=pd.read_csv(reaction_kcat_MW_file,index_col=0)
Concretemodel_Need_Data['reaction_kcat_MW']=unpdate_kcat_mw(Concretemodel_Need_Data['reaction_list'],Concretemodel_Need_Data['reaction_kcat_MW'],'./new_idea_AGGD/EC/reaction_kcat_MW_AGGD.csv')

In [446]:
# Concretemodel_Need_Data['reaction_kcat_MW'].at['A_ADH','kcat_MW']

In [9]:
if __name__ == "__main__":
    pool=ProcessPoolExecutor(max_workers=8)
    with pool as executor:
        start = datetime.datetime.now()
        #print("pass_2")
        tmp=[]
        futures = {executor.submit(fba_max_subs_product,Concretemodel_Need_Data,(list_x[i],list_y[i]),(96,24),0,0,0.40,0.62,'EX_Salid_e'): i for i in range(0,len(list_x))}
        # for future in as_completed(futures):
        #     tmp.append(future.result())
    end = datetime.datetime.now()
    print (end - start)
    print(futures)

0:00:18.896887
{<Future at 0x152484fe208 state=finished returned float>: 0, <Future at 0x152456746a0 state=finished returned float>: 1, <Future at 0x15241ebb828 state=finished returned float>: 2, <Future at 0x15248557438 state=finished returned float>: 3}


In [10]:
result_list_mass=pd.DataFrame()
for future in as_completed(futures):
    x=list_x[futures[future]]
    y=list_y[futures[future]]
    real=1
    real_x=int(x*real)
    real_y=int(y*real)
    print('A:B:C='+str(real_x)+':'+str(real_y))
    print(future.result())
    result_list_mass.at[str(real_x)+':'+str(real_y),'flux']=future.result()
result_list=pd.DataFrame()
for i in range(0,len(list_x)):
    real=1
    real_x=int(list_x[i]*real)
    real_y=int(list_y[i]*real)
    result_list.at[str(real_x)+':'+str(real_y),'flux']=result_list_mass.at[str(real_x)+':'+str(real_y),'flux']
result_list

A:B:C=6:6
3.0741311577743806
A:B:C=4:8
1.2266261232164202
A:B:C=8:4
4.690132708331974
A:B:C=9:3
4.172659144414608


,flux
9:3,4.172659
8:4,4.690133
6:6,3.074131
4:8,1.226626


In [ ]:
print error_print

In [468]:
list_a=[60,80,96,102]
list_b=[60,40,24,18]

### diff_subs

In [469]:
if __name__ == "__main__":
    pool=ProcessPoolExecutor(max_workers=8)
    with pool as executor:
        start = datetime.datetime.now()
        #print("pass_2")
        tmp=[]
        futures = {executor.submit(fba_max_subs_product,Concretemodel_Need_Data,(6,6),(list_a[i],list_b[i]),0,0,0.40,0.62,'EX_Salid_e'): i for i in range(0,len(list_a))}
        # for future in as_completed(futures):
        #     tmp.append(future.result())
    end = datetime.datetime.now()
    print (end - start)
    print(futures)

0:00:39.376847
{<Future at 0x1fb8018e7f0 state=finished raised TypeError>: 0, <Future at 0x1fb85ab09b0 state=finished returned float>: 1, <Future at 0x1fb85ab0438 state=finished returned float>: 2, <Future at 0x1fb8592cef0 state=finished returned float>: 3}


In [470]:
result_list_mass=pd.DataFrame()
for future in as_completed(futures):
    x=list_a[futures[future]]
    y=list_b[futures[future]]
    real=1
    real_x=int(x*real)
    real_y=int(y*real)
    print('glc:xyl='+str(real_x)+':'+str(real_y))
    print(future.result())
    result_list_mass.at[str(real_x)+':'+str(real_y),'flux']=future.result()
result_list=pd.DataFrame()
for i in range(0,len(list_a)):
    real=1
    real_x=int(list_a[i]*real)
    real_y=int(list_b[i]*real)
    result_list.at[str(real_x)+':'+str(real_y),'flux']=result_list_mass.at[str(real_x)+':'+str(real_y),'flux']
result_list

glc:xyl=102:18
3.8306965277756193
glc:xyl=80:40
1.0566235044371814
glc:xyl=96:24
3.0741311577743806
glc:xyl=60:60


TypeError: '>' not supported between instances of 'int' and 'NoneType'

###### iML1515 model/AGGD

In [465]:
model_AGGD=load_json_model(model_file)
model_1515=load_json_model(model_origin_name)

In [466]:
model_AGGD

Name,iML1515
Memory address,0x01fb845c2be0
Number of metabolites,4107
Number of reactions,6154
Objective expression,1.0*A_DM_Tyrosol_c - 1.0*A_DM_Tyrosol_c_reverse_fb37e
Compartments,"cytosol, extracellular space, periplasm"


In [467]:
model_1515

Name,iML1515
Memory address,0x01fb888afd30
Number of metabolites,1877
Number of reactions,2713
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [ ]:
# pass_v=0
# for i in range(0,4):
#     a=ECM_FBA(66,20,list_x[i],list_y[i],0,0,0.15,0.55)
#     print('obj:')
#     print(a.obj())
#     a_obj=get_two_float(a.obj(),2)
#     b=ECM_pFBA(66,20,list_x[i],list_y[i],a_obj,0,0,0.15,0.55)
#     print('pFBA')
#     print('glc:'+str(b.reaction['EX_glc__D_e_reverse'].value))
#     print('glc_A:'+str(b.reaction['A_glc__D_e_con_1'].value))
#     print('glc_B:'+str(b.reaction['B_glc__D_e_con_2'].value))
#     print('xyl:'+str(b.reaction['EX_xyl__D_e_reverse'].value))
#     print('xyl_A:'+str(b.reaction['A_xyl__D_e_con_1_reverse'].value))
#     print('xyl_B:'+str(b.reaction['B_xyl__D_e_con_2'].value))
#     if pass_v>a.obj():
#         print('True')
#     else:
#         print('False')
#     pass_v=a.obj()

obj:
8.474320522667938
pFBA
glc:66.0
glc_A:0.0
glc_B:48.97360273295178
xyl:19.93824121440977
xyl_A:19.93824121440977
xyl_B:0.0
False
obj:
9.459038907498435
pFBA
glc:66.0
glc_A:0.0
glc_B:49.3256507312501
xyl:20.0
xyl_A:20.0
xyl_B:0.0
False
obj:
5.948352142189295
pFBA
glc:66.0
glc_A:0.0
glc_B:59.15827862083334
xyl:20.0
xyl_A:20.0
xyl_B:0.0
True
obj:
1.5785455149663135
pFBA
glc:66.0
glc_A:0.0
glc_B:66.0
xyl:20.0
xyl_A:14.719433488157803
xyl_B:5.280566511842197
True


In [ ]:
def get_pFBA(a,b,x,y,tyr,php):
    fba_result=ECM_FBA(a,b,x,y,tyr,php)
    print("AG:GD="+str(x)+':'+str(y))
    print(fba_result.obj())
    pfba_result=ECM_pFBA(a,b,x,y,fba_result.obj(),tyr,php)
    print('glc：'+str(pfba_result.reaction['EX_glc__D_e_reverse'].value))
    print('xyl：'+str(pfba_result.reaction['EX_xyl__D_e_reverse'].value))
    return pfba_result

In [ ]:
import numpy as np
tyr_a=np.linspace(0.01, 0.1, num=91, endpoint=True, retstep=False, dtype=None)
php_b=np.linspace(0.01, 0.1, num=91, endpoint=True, retstep=False, dtype=None)

In [ ]:
result_tyr_php=pd.DataFrame()

In [ ]:
list_x=[3,2,1,1]
list_y=[1,1,1,2]

In [ ]:
tyr_a=[0.1]
php_b=[0.08]

#### HHHHHHH

In [ ]:
d=ECM_FBA(20,5,1,2,0.01,0.01)
print(d.obj())
e=ECM_FBA(20,5,3,1,0.01,0.01)
print(e.obj())
f=ECM_FBA(20,5,2,1,0.01,0.01)
print(f.obj())
g=ECM_FBA(20,5,1,1,0.01,0.01)
print(g.obj())

0.08620377125475898
1.4260700681163598
1.5363641882054253
1.101838838964568


In [ ]:
e=ECM_FBA(20,5,3,1,0.01,0.01)
print(e.obj())

3.6153776117274674


In [ ]:
e_pfba=ECM_pFBA(20,5,2,1,f.obj(),0.01,0.01)

In [ ]:
print(e_pfba.reaction['EX_glc__D_e_reverse'].value)
print(e_pfba.reaction['EX_xyl__D_e_reverse'].value)
#print(e_pfba.reaction['A_glc__D_e_con_1_reverse'].value)
print(e_pfba.reaction['A_Tyrosol_c2e'].value)
print(e_pfba.reaction['A_Tyrosol_e_con_1'].value)
print(e_pfba.reaction['B_Tyrosol_c2e_reverse'].value)

3.4500963164938825
4.999999999999717
0.28556962517154716
0.5711392503430943
0.5711392503430943


In [ ]:
d.reaction['A_BIOMASS_Ec_iML1515_core_75p37M'].value

0.08

In [ ]:
d.reaction['B_BIOMASS_Ec_iML1515_core_75p37M'].value

0.5

In [ ]:
get_two_float(d.obj(),2)

3.86

In [ ]:
d_pfba=ECM_pFBA(20,5,1,2,d.obj(),0,0)

In [ ]:
print(d_pfba.reaction['EX_glc__D_e_reverse'].value)
print(d_pfba.reaction['EX_xyl__D_e_reverse'].value)
print(d_pfba.reaction['A_glc__D_e_con_1_reverse'].value)
print(d_pfba.reaction['A_Tyrosol_c2e'].value)
print(d_pfba.reaction['A_Tyrosol_e_con_1'].value)
print(d_pfba.reaction['B_Tyrosol_c2e_reverse'].value)

20.0
4.999999999998122
0.0
0.12647389334847503
0.12647389334847503
0.06323694667423752


In [ ]:
a=0
for i in tyr_a:
    for j in php_b:
        for k in range(4):
            c=ECM_FBA(24,6,list_x[k],list_y[k],i,j)
            result_tyr_php.at[a,'tyr:php']=str(i)+':'+str(j)
            result_tyr_php.at[a,'AG:GD='+str(list_x[k])+':'+str(list_y[k])]=c.obj()
            c_pfba=ECM_pFBA(24,6,list_x[k],list_y[k],c.obj(),i,j)
            result_tyr_php.at[a,'No.'+str(k+1)+'glc']=c_pfba.reaction['EX_glc__D_e_reverse'].value
            result_tyr_php.at[a,'No.'+str(k+1)+'xyl']=c_pfba.reaction['EX_xyl__D_e_reverse'].value
        print(result_tyr_php.loc[a])
        print('---------------------')
        a=a+1

In [ ]:
result_tyr_php.to_csv('./new_idea_AGGD/result/result_tyr_php.csv')

In [ ]:
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

#### LLLLLLLLLL

In [ ]:
d=ECM_FBA(30,5,1,2,0.09,0)
print(d.obj())
e=ECM_FBA(30,5,2,1,0.09,0)
print(e.obj())
f=ECM_FBA(30,5,1,1,0.09,0)
print(f.obj())
g=ECM_FBA(30,5,1,4,0.09,0)
print(g.obj())
h=ECM_FBA(30,5,1,6,0.09,0)
print(h.obj())

1.0968870330466416
1.7724695743337406
1.1018265468119353
    model=unknown;
        message from solver=Problem proven to be infeasible or unbounded.
None
    model=unknown;
        message from solver=Problem proven to be infeasible or unbounded.
None


In [ ]:
pass_v=0
for i in range(0,5):
    a=ECM_FBA(100,list_x[i],list_y[i],0,0,0.1,0.1)
    print('obj:')
    print(a.obj())
    a_obj=get_two_float(a.obj(),2)
    b=ECM_pFBA(100,list_x[i],list_y[i],a_obj,0,0,0.1,0.1)
    print('pFBA')
    print('glc:'+str(b.reaction['EX_glc__D_e_reverse'].value))
    print('glc_A:'+str(b.reaction['A_glc__D_e_con_1_reverse'].value))
    print('glc_B:'+str(b.reaction['B_glc__D_e_con_2'].value))
    if pass_v>a.obj():
        print('True')
    else:
        print('False')
    pass_v=a.obj()

### C AG:GD=3:1

In [ ]:
c=get_pFBA(24,6,9,3)

AG:GD=3:1
0.02127210537787505
glc：20.2803587732229
xyl：6.0


In [ ]:
c.reaction['EX_glc__D_e_reverse'].value

20.2803587732229

### D AG：GD=2:1

In [ ]:
d=get_pFBA(24,6,8,4)

AG:GD=2:1
0.3773713110559811
glc：23.194871815772935
xyl：4.789821164688448


### E AG:GD=1:1

In [ ]:
e=get_pFBA(24,6,6,6)

AG:GD=1:1
0.37737131105552635
glc：23.194871815653045
xyl：2.877118993967035


### AG:GD=1:2

In [ ]:
f=get_pFBA(100,100,4,8)

AG:GD=1:2
4.091261615233407
    model=unknown;
        message from solver=Model was proven to be infeasible.
glc：None
xyl：None


In [ ]:
f.reaction['A_tyr__L_e_con_1'].value

0.1

In [ ]:
f.reaction['B_phpyr_e_con_2_reverse'].value

0.07

In [ ]:
f.obj()==e.obj()

False

In [ ]:
for m in range(1,10):
    n=100-m
    try:
        a=ECM_FBA(100,100,m,n)
        result_ratio.at[m,'S_value']=a.obj()*n
    except:
        print(str(m)+':error')
        continue
    # if (a.reaction['A_Tyrosol_e_con_1'].value-a.reaction['A_Tyrosol_e_con_1_reverse'].value)>=0 or (a.reaction['A_tyr__L_e_con_1'].value-a.reaction['A_tyr__L_e_con_1_reverse'].value)<=0\
    #      or (a.reaction['A_phpyr_e_con_1'].value-a.reaction['A_phpyr_e_con_1_reverse'].value)<=0:
    #     pass
    result_ratio.at[m,'A']=m
    result_ratio.at[m,'B']=n
    result_ratio.at[m,'obj']=a.obj()
    result_ratio.at[m,'S_value']=a.obj()*n
    result_ratio.at[m,'A_glc_out']=a.reaction['A_glc__D_e_con_1'].value
    result_ratio.at[m,'B_glc']=a.reaction['B_glc__D_e_con_2'].value-a.reaction['B_glc__D_e_con_2_reverse'].value
    result_ratio.at[m,'A_xyl']=a.reaction['A_xyl__D_e_con_1_reverse'].value-a.reaction['A_xyl__D_e_con_1'].value
    result_ratio.at[m,'B_xyl']=a.reaction['B_xyl__D_e_con_2'].value-a.reaction['B_xyl__D_e_con_2_reverse'].value
    result_ratio.at[m,'A_c2e']=a.reaction['A_Tyrosol_c2e'].value-a.reaction['A_Tyrosol_c2e_reverse'].value
    result_ratio.at[m,'T_A_value']=a.reaction['A_Tyrosol_e_con_1'].value
    result_ratio.at[m,'T_B_value']=a.reaction['B_Tyrosol_e_con_2'].value
    result_ratio.at[m,'B_e2c']=a.reaction['B_Tyrosol_c2e_reverse'].value-a.reaction['B_Tyrosol_c2e'].value
    print(str(a.obj()*n),m,result_ratio.at[m,'A_c2e'],result_ratio.at[m,'T_A_value'],result_ratio.at[m,'T_B_value'],result_ratio.at[m,'B_e2c'])

In [ ]:
b=ECM_pFBA(200,a.obj())

In [ ]:
a.reaction['B_R00271'].value

In [ ]:
a.reaction['A_akg_e_con_1_reverse'].value

In [ ]:
a.reaction['A_akg_e_con_1'].value

In [ ]:
a.reaction['B_akg_e_con_2_reverse'].value

In [ ]:
a.reaction['B_akg_e_con_2'].value

In [ ]:
a=ECM_FBA_2(200,3,97)

In [ ]:
from cobra.io import load_json_model

In [ ]:
model_ab=load_json_model(model_file)
cobra.manipulation.modify.convert_to_irreversible(model_ab)
isoenzyme_split(model_ab)

In [ ]:
xyl_glc_ECM=pd.DataFrame()
for i in Concretemodel_Need_Data['reaction_list']:
    try:
        xyl_glc_ECM.at[i,'flux']=b.reaction[i].value
        xyl_glc_ECM.at[i,'kcat_MW']=Concretemodel_Need_Data['reaction_kcat_MW'].at[i,'kcat_MW']
        xyl_glc_ECM.at[i,'enz_useage']=round(xyl_glc_ECM.at[i,'flux']/xyl_glc_ECM.at[i,'kcat_MW'],6)
    except:
        pass
    xyl_glc_ECM.at[i,'formula']=model_ab.reactions.get_by_id(i).reaction
xyl_glc_ECM.to_csv('./new_idea/result/2.csv')

In [ ]:
xyl_glc_ECM=pd.DataFrame()
for i in Concretemodel_Need_Data['reaction_list']:
    try:
        xyl_glc_ECM.at[i,'flux']=d.reaction[i].value
        xyl_glc_ECM.at[i,'kcat_MW']=Concretemodel_Need_Data['reaction_kcat_MW'].at[i,'kcat_MW']
        xyl_glc_ECM.at[i,'enz_useage']=round(xyl_glc_ECM.at[i,'flux']/xyl_glc_ECM.at[i,'kcat_MW'],6)
    except:
        pass
    xyl_glc_ECM.at[i,'formula']=model_ab.reactions.get_by_id(i).reaction
xyl_glc_ECM.to_csv('./new_idea/result/3.csv')

In [ ]:
a.obj()

In [ ]:
b.obj()

In [ ]:
a.reaction['R05099'].value

In [ ]:
c=ECM_pFBA3(200,9.84)

In [ ]:
c.obj()